# Creating a live dashboard using Twitch API

+ Twitch API let's you access live game, stream, and user-level data. In this project, we are trying to analyze the viewers for top streamers and how this variation changes through the factors like game, language, and country.
+ Initial ideas are to create a live visualization that displays game level data, and top streamers within that.

- Vardayini, Maddhujeet, and Kaivalya

## Learning to access Twitch API

### Importing libraries

In [1]:
import json
import requests
import pandas as pd
from pandas import json_normalize
import time
import threading

### Acquiring access code from twitch using Client ID & Client Secret

In [2]:

client_id = '0rwk25285ozd8rrb1a5ig23a5r2wae'
client_secret = 'pirkkzb4uxf4z0hmkw1e9i3f49cf2w'

access_code = requests.post('https://id.twitch.tv/oauth2/token?client_id='+str(client_id)+'&client_secret='+str(client_secret)+'&grant_type=client_credentials')
           

In [3]:
token = json.loads(access_code.text)
access_token = token['access_token']

In [4]:
headers = {
    'Authorization' : 'Bearer '+str(access_token),
    'Client-ID': client_id,
}
response = requests.get('https://api.twitch.tv/helix/streams?first=100', headers=headers)

In [5]:
print(response.text)
response_json = json.loads(response.text)
print(response_json)

{"data":[{"id":"41908496216","user_id":"90075649","user_login":"illojuan","user_name":"IlloJuan","game_id":"509658","game_name":"Just Chatting","type":"live","title":"EL CABRÓN 🐐 GRACIAS ❤️","viewer_count":48071,"started_at":"2024-02-19T20:22:35Z","language":"es","thumbnail_url":"https://static-cdn.jtvnw.net/previews-ttv/live_user_illojuan-{width}x{height}.jpg","tag_ids":[],"tags":["Español"],"is_mature":false},{"id":"42328762985","user_id":"65171890","user_login":"rainbow6","user_name":"Rainbow6","game_id":"460630","game_name":"Tom Clancy's Rainbow Six Siege","type":"live","title":"Six Invitational 2024 - Playoffs Day 1 - Spacestation Gaming vs DarkZero Esports","viewer_count":42103,"started_at":"2024-02-19T15:58:23Z","language":"en","thumbnail_url":"https://static-cdn.jtvnw.net/previews-ttv/live_user_rainbow6-{width}x{height}.jpg","tag_ids":[],"tags":["English","Esports","ubisoft","R6S","rainbowsixsiege","DropsEnabled"],"is_mature":true},{"id":"42328153705","user_id":"181077473","use

In [6]:
data = response_json['data']

df = pd.DataFrame.from_dict(json_normalize(data), orient='columns')

In [7]:
df.head()

,id,user_id,user_login,user_name,game_id,game_name,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids,tags,is_mature
0,41908496216,90075649,illojuan,IlloJuan,509658,Just Chatting,live,EL CABRÓN 🐐 GRACIAS ❤️,48071,2024-02-19T20:22:35Z,es,https://static-cdn.jtvnw.net/previews-ttv/live...,[],[Español],False
1,42328762985,65171890,rainbow6,Rainbow6,460630,Tom Clancy's Rainbow Six Siege,live,Six Invitational 2024 - Playoffs Day 1 - Space...,42103,2024-02-19T15:58:23Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[English, Esports, ubisoft, R6S, rainbowsixsie...",True
2,42328153705,181077473,gaules,Gaules,32399,Counter-Strike,live,O Plano vs SALAO DO CORTE IEM Dallas 2024 Sout...,36384,2024-02-19T10:37:16Z,pt,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[Português, fps, brasil, brazil, Portugues, co...",True
3,43655036843,36340781,tarik,tarik,516575,VALORANT,live,MIBR vs FURIA - VCT 2024: Americas Kickoff - T...,35292,2024-02-19T21:51:31Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],[English],False
4,50441180477,94753024,mizkif,Mizkif,509658,Just Chatting,live,GOING TO COMPETTING GYMS TO BRING IN NEW CUSTO...,29354,2024-02-19T20:53:08Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[ADHD, Osteoporosis, OTK, STARFORGE, STEROIDS,...",False


### Creating a CSV for Stream data

In [8]:
export_csv = df.to_csv (r'D:/UTD/Notebooks/Twitch/first.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path


# Saved to local directory:/Users/kaivalya/Twitch/ as first.csv

## Accessing Game data 

### Making a list of all games in the top 100 streams

In [9]:
game_ids = list(df['game_id'])
game_ids = '&id='.join(game_ids)

In [10]:
game_ids

'509658&id=460630&id=32399&id=516575&id=509658&id=940188387&id=509658&id=509658&id=516575&id=509658&id=32982&id=21779&id=32982&id=32982&id=502732&id=737987407&id=1952699919&id=509658&id=32982&id=18846&id=21779&id=33214&id=766571430&id=516575&id=516575&id=516575&id=1281851716&id=32982&id=21779&id=32982&id=516575&id=509658&id=32982&id=21779&id=32399&id=509658&id=27471&id=516575&id=516575&id=21842902&id=745589378&id=512980&id=491931&id=32982&id=491931&id=116747788&id=32982&id=143106037&id=511224&id=1952699919&id=516575&id=516575&id=32982&id=29595&id=26936&id=538054672&id=460630&id=509658&id=32982&id=516575&id=2091165871&id=493057&id=516575&id=95777581&id=512710&id=32982&id=766571430&id=323323854&id=509658&id=21779&id=209738946&id=33214&id=32982&id=491487&id=460630&id=209738946&id=27546&id=460630&id=509658&id=513143&id=21779&id=509658&id=509658&id=32982&id=33214&id=493887283&id=27471&id=18846&id=27471&id=516575&id=1952699919&id=21779&id=509658&id=511224&id=1337444628&id=516575'

### Passing the list of 100 games to get the game's name from its ID

In [11]:
headers = {
    'Authorization' : 'Bearer '+str(access_token),
    'Client-ID': client_id,
}
response_games = requests.get('https://api.twitch.tv/helix/games?id='+game_ids, headers=headers)

response_games_json = json.loads(response_games.text)

### Converting game data json into a dataframe

In [12]:
data_games = response_games_json['data']

games_df = pd.DataFrame.from_dict(json_normalize(data_games), orient='columns')
games_df.head()

,id,name,box_art_url,igdb_id
0,26936,Music,https://static-cdn.jtvnw.net/ttv-boxart/26936-...,
1,33214,Fortnite,https://static-cdn.jtvnw.net/ttv-boxart/33214-...,1905
2,511224,Apex Legends,https://static-cdn.jtvnw.net/ttv-boxart/511224...,114795
3,116747788,"Pools, Hot Tubs, and Beaches",https://static-cdn.jtvnw.net/ttv-boxart/116747...,
4,209738946,Wuthering Waves,https://static-cdn.jtvnw.net/ttv-boxart/209738...,202956


### Converting the df into a csv to explore in Tableau

In [13]:
export_games_csv = games_df.to_csv (r'D:/UTD/Notebooks/Twitch/first_games.csv', index = None, header=True) #Don't forget to add '.csv' at the end of the path

# Saved in local directory: /Users/kaivalya/Twitch/ as first_games.csv

### repeatable code

In [156]:

def twitch():
    threading.Timer(60.0, twitch).start()
    
    headers = {
        'Authorization' : 'Bearer '+str(access_token),
        'Client-ID': client_id,
    }
    response = requests.get('https://api.twitch.tv/helix/streams?first=100', headers=headers)

    response_json = json.loads(response.text)
    data = response_json['data']
    df = pd.DataFrame.from_dict(json_normalize(data), orient='columns')
    export_csv = df.to_csv (r'D:/UTD/Notebooks/Twitch/first.csv', index = None, header=True)
    
    game_ids = list(df['game_id'])
    game_ids = '&id='.join(game_ids)

    headers = {
        'Authorization' : 'Bearer '+str(access_token),
        'Client-ID': client_id,
    }
    response_games = requests.get('https://api.twitch.tv/helix/games?id='+game_ids, headers=headers)

    #print(response_games.text)
    response_games_json = json.loads(response_games.text)
    data_games = response_games_json['data']
    games_df = pd.DataFrame.from_dict(json_normalize(data_games), orient='columns')
    export_games_csv = games_df.to_csv (r'D:/UTD/Notebooks/Twitch/first_games.csv', index = None, header=True)

twitch()

## Exploring other APIs

### Getting Streamers information

In [14]:
user_ids = list(df['user_id'])
user_ids = '&id='.join(user_ids)
user_ids

'90075649&id=65171890&id=181077473&id=36340781&id=94753024&id=52130765&id=77827128&id=552120296&id=598903130&id=207813352&id=531177917&id=51496027&id=163836275&id=109492660&id=59799994&id=12875057&id=23161357&id=198633200&id=155601320&id=27115917&id=85397463&id=233300375&id=23735582&id=96116107&id=499928989&id=502014446&id=26301881&id=431146848&id=140772558&id=267635380&id=128002336&id=79769388&id=95873995&id=87183799&id=1039034874&id=106820088&id=3389768&id=128775333&id=135468063&id=38244180&id=28575692&id=111799555&id=427632467&id=38746172&id=26490481&id=935309911&id=121510236&id=1895664&id=123182260&id=40972890&id=430023505&id=47119647&id=122808988&id=40488774&id=262319971&id=90222258&id=68727743&id=821717189&id=13884994&id=66418614&id=40063341&id=39154501&id=75891532&id=53065331&id=120244187&id=173066877&id=128440061&id=26921830&id=43548655&id=24991333&id=100901794&id=272748387&id=41875213&id=61812950&id=132106826&id=152633332&id=30623831&id=246244003&id=246450563&id=60672785&id=59

In [15]:
headers = {
    'Authorization': 'Bearer '+str(access_token),
    'Client-ID': client_id,
}
response_user_ids = requests.get('https://api.twitch.tv/helix/users?id='+user_ids, headers=headers)

In [16]:
response_user_json = json.loads(response_user_ids.text)
data_user = response_user_json['data']
users_df = pd.DataFrame.from_dict(json_normalize(data_user), orient='columns')

users_df.head()

,id,login,display_name,type,broadcaster_type,description,profile_image_url,offline_image_url,view_count,created_at
0,65171890,rainbow6,Rainbow6,,partner,Rainbow Six Esports official broadcast for glo...,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,0,2014-06-27T13:51:55Z
1,181077473,gaules,Gaules,,partner,Mais um guerreiro da Maior Tribo do Mundo! Atu...,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,0,2017-11-09T14:10:52Z
2,36340781,tarik,tarik,,partner,ayo.,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,0,2012-09-21T20:43:25Z
3,94753024,mizkif,Mizkif,,partner,"I stream every Monday, Wednesday, Friday, And ...",https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,0,2015-06-28T22:35:16Z
4,52130765,squeezie,Squeezie,,partner,ça live chaque mercredi de 20h à 00h,https://static-cdn.jtvnw.net/jtv_user_pictures...,https://static-cdn.jtvnw.net/jtv_user_pictures...,0,2013-11-24T21:40:16Z


In [17]:
users_df['broadcaster_type'].value_counts()

broadcaster_type
partner      92
              2
affiliate     1
Name: count, dtype: int64

## pagination & game data

In [18]:
headers = {
    'Authorization' : 'Bearer '+str(access_token),
    'Client-ID': client_id,
}
games_response = requests.get('https://api.twitch.tv/helix/games/top?first=100', headers=headers)

In [19]:
#print(response.text)
games_response_json = json.loads(games_response.text)

topgames_data = games_response_json['data']

topgames_df = pd.DataFrame.from_dict(json_normalize(topgames_data), orient='columns')
topgames_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           100 non-null    object
 1   name         100 non-null    object
 2   box_art_url  100 non-null    object
 3   igdb_id      100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [20]:
topgames_df.head()

,id,name,box_art_url,igdb_id
0,509658,Just Chatting,https://static-cdn.jtvnw.net/ttv-boxart/509658...,
1,32982,Grand Theft Auto V,https://static-cdn.jtvnw.net/ttv-boxart/32982_...,1020
2,516575,VALORANT,https://static-cdn.jtvnw.net/ttv-boxart/516575...,126459
3,21779,League of Legends,https://static-cdn.jtvnw.net/ttv-boxart/21779-...,115
4,32399,Counter-Strike,https://static-cdn.jtvnw.net/ttv-boxart/32399-...,


In [21]:
topgames_data = games_response_json['pagination']
pagination= topgames_data['cursor']
print(pagination)
print(topgames_data)
str(pagination)

eyJzIjoxMDAsImQiOmZhbHNlLCJ0Ijp0cnVlfQ==
{'cursor': 'eyJzIjoxMDAsImQiOmZhbHNlLCJ0Ijp0cnVlfQ=='}


'eyJzIjoxMDAsImQiOmZhbHNlLCJ0Ijp0cnVlfQ=='

In [22]:
topgames_data = games_response_json['pagination']
pagination= topgames_data['cursor']
print(pagination)
print(topgames_data)
str(pagination)

headers = {
    'Authorization' : 'Bearer '+str(access_token),
    'Client-ID': client_id,
}
games_response = requests.get('https://api.twitch.tv/helix/games/top?first=100&after='+str(pagination), headers=headers)

eyJzIjoxMDAsImQiOmZhbHNlLCJ0Ijp0cnVlfQ==
{'cursor': 'eyJzIjoxMDAsImQiOmZhbHNlLCJ0Ijp0cnVlfQ=='}


In [23]:
#print(response.text)
games_response_json = json.loads(games_response.text)

topgames_data = games_response_json['data']
topgames_df_temp = pd.DataFrame.from_dict(json_normalize(topgames_data), orient='columns')
topgames_df_temp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           100 non-null    object
 1   name         100 non-null    object
 2   box_art_url  100 non-null    object
 3   igdb_id      100 non-null    object
dtypes: object(4)
memory usage: 3.3+ KB


In [24]:
print(games_response)

<Response [200]>


In [25]:
#topgames_df.append(topgames_df_temp, ignore_index=True, sort=False)
frames = [topgames_df, topgames_df_temp]
topgames_df = pd.concat(frames, ignore_index=True)
topgames_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           200 non-null    object
 1   name         200 non-null    object
 2   box_art_url  200 non-null    object
 3   igdb_id      200 non-null    object
dtypes: object(4)
memory usage: 6.4+ KB


In [27]:
export_topgames_csv = topgames_df.to_csv (r'D:/UTD/Notebooks/Twitch/topgames.csv', index = None, header=True)

### getting stream data for the games

In [28]:
topstreamsforgame_ids = topgames_df['id']
topstreamsforgame_ids[0]

'509658'

In [29]:
headers = {
    'Authorization' : 'Bearer '+str(access_token),
    'Client-ID': client_id,
}
topstreamsforgame_response = requests.get('https://api.twitch.tv/helix/streams?game_id='+str(topgames_df['id'][0])+'&first=100', headers=headers)


In [30]:
print(topstreamsforgame_response)

<Response [200]>


In [31]:
topstreamsforgame_response_json = json.loads(topstreamsforgame_response.text)

topstreamsforgame_data = topstreamsforgame_response_json['data']

topstreamsforgame_df = pd.DataFrame.from_dict(json_normalize(topstreamsforgame_data), orient='columns')
topstreamsforgame_df.head()

,id,user_id,user_login,user_name,game_id,game_name,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids,tags,is_mature
0,41908496216,90075649,illojuan,IlloJuan,509658,Just Chatting,live,EL CABRÓN 🐐 GRACIAS ❤️,48196,2024-02-19T20:22:35Z,es,https://static-cdn.jtvnw.net/previews-ttv/live...,[],[Español],False
1,50441180477,94753024,mizkif,Mizkif,509658,Just Chatting,live,GOING TO COMPETTING GYMS TO BRING IN NEW CUSTO...,28037,2024-02-19T20:53:08Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[ADHD, Osteoporosis, OTK, STARFORGE, STEROIDS,...",False
2,50441244317,552120296,zackrawrr,zackrawrr,509658,Just Chatting,live,BIG DAY--NEWS/REACTS/GAMES--WUTHERING WAVES ...,27081,2024-02-19T21:01:08Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[English, OTK, otknetwork, StarForgePCs, wow, ...",False
3,41908401192,77827128,tumblurr,Tumblurr,509658,Just Chatting,live,KING IS ALIVE MEGA DRAMA ALERT | !vpn,25502,2024-02-19T20:06:20Z,it,https://static-cdn.jtvnw.net/previews-ttv/live...,[],[Italiano],False
4,43653906155,207813352,hasanabi,HasanAbi,509658,Just Chatting,live,🚨MONDAY NEWSDAY🚨GENZ CORRESPONDENT @SEANDABLAC...,20585,2024-02-19T19:25:06Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[news, politics, adhd, English]",False


### Automating the code pull

Automating the pulling of top streamer data for top games. I put in a sleep timer for 30 seconds, so this snippet takes about 20 minutes to run depending on the recurrence of for loops

In [32]:
# REALLY LONG CODE
#Accessing API
headers = {
    'Authorization' : 'Bearer '+str(access_token),
    'Client-ID': client_id,
}
topstreamsforgame_response = requests.get('https://api.twitch.tv/helix/streams?game_id='+str(topgames_df['id'][0])+'&first=25', headers=headers)

# Converting to JSON
topstreamsforgame_response_json = json.loads(topstreamsforgame_response.text)

# Getting data from the JSON
topstreamsforgame_data = topstreamsforgame_response_json['data']

# Converting into a DataFrame
topstreamsforgame_df = pd.DataFrame.from_dict(json_normalize(topstreamsforgame_data), orient='columns')

# Automated flow
for i in range(1,19) :
    
    headers = {
        'Authorization' : 'Bearer '+str(access_token),
        'Client-ID': client_id,
    }
    topstreamsforgame_response = requests.get('https://api.twitch.tv/helix/streams?game_id='+str(topgames_df['id'][i])+'&first=25', headers=headers)
    
    topstreamsforgame_response_json = json.loads(topstreamsforgame_response.text)
    topstreamsforgame_data = topstreamsforgame_response_json['data']
    topstreamsforgame_df_temp = pd.DataFrame.from_dict(json_normalize(topstreamsforgame_data), orient='columns')
    
    frames = [topstreamsforgame_df, topstreamsforgame_df_temp]
    topstreamsforgame_df = pd.concat(frames, ignore_index=True)
    
    #time.sleep(30)

topstreamsforgame_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 464 entries, 0 to 463
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   id             464 non-null    object
 1   user_id        464 non-null    object
 2   user_login     464 non-null    object
 3   user_name      464 non-null    object
 4   game_id        464 non-null    object
 5   game_name      464 non-null    object
 6   type           464 non-null    object
 7   title          464 non-null    object
 8   viewer_count   464 non-null    int64 
 9   started_at     464 non-null    object
 10  language       464 non-null    object
 11  thumbnail_url  464 non-null    object
 12  tag_ids        464 non-null    object
 13  tags           463 non-null    object
 14  is_mature      464 non-null    bool  
dtypes: bool(1), int64(1), object(13)
memory usage: 51.3+ KB


In [42]:
topstreamsforgame_df.head()

,id,user_id,user_login,user_name,game_id,game_name,type,title,viewer_count,started_at,language,thumbnail_url,tag_ids,tags,is_mature
0,41908496216,90075649,illojuan,IlloJuan,509658,Just Chatting,live,EL CABRÓN 🐐 GRACIAS ❤️,48196,2024-02-19T20:22:35Z,es,https://static-cdn.jtvnw.net/previews-ttv/live...,[],[Español],False
1,50441180477,94753024,mizkif,Mizkif,509658,Just Chatting,live,GOING TO COMPETTING GYMS TO BRING IN NEW CUSTO...,28037,2024-02-19T20:53:08Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[ADHD, Osteoporosis, OTK, STARFORGE, STEROIDS,...",False
2,50441244317,552120296,zackrawrr,zackrawrr,509658,Just Chatting,live,BIG DAY--NEWS/REACTS/GAMES--WUTHERING WAVES ...,27081,2024-02-19T21:01:08Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[English, OTK, otknetwork, StarForgePCs, wow, ...",False
3,41908401192,77827128,tumblurr,Tumblurr,509658,Just Chatting,live,KING IS ALIVE MEGA DRAMA ALERT | !vpn,25502,2024-02-19T20:06:20Z,it,https://static-cdn.jtvnw.net/previews-ttv/live...,[],[Italiano],False
4,43653906155,207813352,hasanabi,HasanAbi,509658,Just Chatting,live,🚨MONDAY NEWSDAY🚨GENZ CORRESPONDENT @SEANDABLAC...,20585,2024-02-19T19:25:06Z,en,https://static-cdn.jtvnw.net/previews-ttv/live...,[],"[news, politics, adhd, English]",False


In [34]:
export_topstreamsforgame_csv = topstreamsforgame_df.to_csv (r'D:/UTD/Notebooks/Twitch/topstreamsforgame.csv', index = None, header=True)